# Fireveg DB - plot of fire history per sampling location

Author: [José R. Ferrer-Paris](https://github.com/jrfep) and [Ada Sánchez-Mercado](https://github.com/adasanchez)

Date: July 2024

This Jupyter Notebook includes R code to check taxonomic alignment for species names used in the Fireveg Database. 

The input is loaded from a public data record of the database.


The animal and plant lists in BioNet Atlas are maintained by the BioNet Data Team.

For taxonomic decisions on plants, BioNet uses the following sources in order of precedence (eg if there is inconsistency between the NSW Biodiversity Conservation Act 2016 and the information in PlantNet then BioNet names will be consistent with the former):
 - NSW Biodiversity Conservation Act 2016 https://legislation.nsw.gov.au/view/html/inforce/current/act-2016-063
 - PlantNet https://plantnet.rbgsyd.nsw.gov.au/search/simple.htm
 - APNI https://www.anbg.gov.au/apni/
 - IPNI https://www.ipni.org/

[BIONET species names web standard](https://www.environment.nsw.gov.au/-/media/OEH/Corporate-Site/Documents/BioNet/bionet-species-names-web-service-data-standard-v-1-2-220050.pdf)

## Set-up
### Load libraries


In [1]:
library(ggplot2)
##library(forcats)
library(dplyr)
#library(data.table)
require(tidyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr



### Working directory

In [2]:
here::i_am("taxonomic-coverage/Check-taxonomy.ipynb")

here() starts at /Users/z3529065/proyectos/fireveg/fireveg-analysis



### Load data
Check if a copy of the data is available in the local data folder, otherwise download from OSF cloud drive. 

In [3]:
data_dir <- here::here("data")
if (!dir.exists(data_dir))
    dir.create(data_dir)

for (RDSfile in c("Summary-traits-species.rds")) {
    RDSpath <- here::here(data_dir, RDSfile)
    if (file.exists(RDSpath)) {
        cat(sprintf("RDS file found at:\n%s\nNo need to download!\n\n", RDSpath))
    } else {
        require(osfr)
        osf_project <- osf_retrieve_node("https://osf.io/h96q2")
        file_list <- osf_ls_files(osf_project, pattern=RDSfile)
        osf_download(file_list,
                 data_dir,
                 conflicts = "overwrite")
    }
}

RDS file found at:
/Users/z3529065/proyectos/fireveg/fireveg-analysis/data/Summary-traits-species.rds
No need to download!



In [4]:
#quadrats_table <- readRDS(here::here(data_dir,"Quadrat-sample-data.rds"))
all_traits <- readRDS(here::here(data_dir,"Summary-traits-species.rds"))


In [5]:
glimpse(all_traits)

Rows: 15,732
Columns: 24
$ family          <chr> "Brassicaceae", "Myrtaceae", "Myrtaceae", "Apiaceae", …
$ genus           <chr> "Lepidium", "Eucalyptus", "Melaleuca", "Actinotus", "A…
$ spp             <dbl> 2358, 2359, 2360, 2361, 2362, 2363, 2364, 2365, 2366, …
$ species         <chr> "Lepidium oxytrichum", "Eucalyptus williamsiana", "Mel…
$ current_spp     <dbl> 2358, 2359, 2360, 2361, 2362, 2363, 22952, 2365, 2366,…
$ current_species <chr> "Lepidium oxytrichum", "Eucalyptus williamsiana", "Mel…
$ taxonrank       <chr> "Species", "Species", "Species", "Species", "Species",…
$ establishment   <chr> "Alive in NSW, Native", "Alive in NSW, Native", "Alive…
$ current         <chr> "true", "true", "true", "true", "true", "true", "false…
$ nquadrat        <dbl> 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 15, 0, 0, 0, 0, 0, 0, 8,…
$ germ8           <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, …
$ rect2           <dbl> 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, …
$ germ1        

## Check taxonomic alignment with APCalign

From Will Cornwell:

> APC will apparently in the medium-term future going to a formal version control, but in the meantime we've been working with Anne Fuchs and Anna Munro to get a dynamic updating system here:  https://www.publish.csiro.au/BT/pdf/BT24014.
> 
>  Somewhere in the flow from RBG -> BioNET -> AVH -> ALA, there is a taxonomy switch. AVH (Australian Virtual Herbarium) seems to be APC, so I guess it's either it's on export from BioNET or input into AVH?
 
> I checked Palmeria racemosa which is for some reason not recognized by NSW, and it's not in the database, so it does seem like PlantNET taxonomy. So maybe check what's happening to names not in PlantNET?

Following recommendations from:
https://traitecoevo.github.io/APCalign/articles/APCalign.html

In [6]:
library(APCalign)

Download the stable version of the taxonomic resources

In [7]:
stable_resources <- load_taxonomic_resources(stable_or_current_data = "stable")


Loading resources into memory...



...done



In [22]:
fireveg_species_names <- all_traits |> 
  distinct(species) |>
  pull(species) |>
  create_taxonomic_update_lookup(resources = stable_resources)

Checking alignments of 15648 taxa


  -> of these 9044 names have a perfect match to a scientific name in the APC. 
      Alignments being sought for remaining names.



In [23]:

fireveg_species_names |> 
  print(n = 6)

# A tibble: 15,648 × 12
  original_name       aligned_name accepted_name suggested_name genus taxon_rank
  <chr>               <chr>        <chr>         <chr>          <chr> <chr>     
1 Lepidium oxytrichum Lepidium ox… Lepidium oxy… Lepidium oxyt… Lepi… species   
2 Eucalyptus william… Eucalyptus … Eucalyptus w… Eucalyptus wi… Euca… species   
3 Melaleuca glomerata Melaleuca g… Melaleuca gl… Melaleuca glo… Mela… species   
4 Actinotus helianthi Actinotus h… Actinotus he… Actinotus hel… Acti… species   
5 Apium prostratum    Apium prost… Apium prostr… Apium prostra… Apium species   
6 Cryptocarya obovata Cryptocarya… Cryptocarya … Cryptocarya o… Cryp… species   
# ℹ 15,642 more rows
# ℹ 6 more variables: taxonomic_dataset <chr>, taxonomic_status <chr>,
#   scientific_name <chr>, aligned_reason <chr>, update_reason <chr>,
#   number_of_collapsed_taxa <dbl>


In [27]:
fireveg_species_names |> 
    group_by(update_reason,taxon_rank) |> 
    summarise(names = n_distinct(original_name), .groups = "drop") |>
    pivot_wider(names_from = taxon_rank, values_from = names)

update_reason,form,species,subspecies,variety,family,genus,NA
<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
aligned name accepted by APC,28,8259,811,379,NA,NA,NA
basionym,NA,137,20,13,NA,NA,NA
doubtful taxonomic synonym,2,8,2,1,NA,NA,NA
excluded,3,96,4,1,NA,NA,NA
misapplied,NA,132,6,8,NA,NA,NA
nomenclatural synonym,4,1125,211,261,NA,NA,NA
orthographic variant,NA,139,6,15,NA,NA,NA
pro parte misapplied,NA,36,1,NA,NA,NA,NA
pro parte nomenclatural synonym,NA,NA,1,1,NA,NA,NA


In [11]:
fireveg_species_names |> filter(update_reason %in% "pro parte misapplied") |> distinct()

original_name,aligned_name,accepted_name,suggested_name,genus,taxon_rank,taxonomic_dataset,taxonomic_status,scientific_name,aligned_reason,update_reason,number_of_collapsed_taxa
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Myriophyllum elatinoides,Myriophyllum elatinoides,Myriophyllum caput-medusae,Myriophyllum caput-medusae [alternative possible names: Myriophyllum porcatum (pro parte misapplied) | Myriophyllum salsugineum (pro parte misapplied) | Myriophyllum triphyllum (pro parte misapplied)],Myriophyllum,species,APC,accepted,Myriophyllum caput-medusae Orchard,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Xanthium strumarium,Xanthium strumarium,Xanthium cavanillesii,Xanthium cavanillesii [alternative possible names: Xanthium italicum (pro parte misapplied) | Xanthium occidentale (pro parte misapplied) | Xanthium orientale (pro parte misapplied)],Xanthium,species,APC,accepted,Xanthium cavanillesii Schouw,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Mischocarpus sundaicus,Mischocarpus sundaicus,Mischocarpus australis,Mischocarpus australis [alternative possible names: Mischocarpus macrocarpus (pro parte misapplied) | Mischocarpus stipitatus (pro parte misapplied) | Mischocarpus australis (misapplied)],Mischocarpus,species,APC,accepted,Mischocarpus australis S.T.Reynolds,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Rubus fruticosus,Rubus fruticosus,Rubus anglocandicans,Rubus anglocandicans [alternative possible names: Rubus erythrops (pro parte misapplied) | Rubus laciniatus (pro parte misapplied) | Rubus leucostachys (pro parte misapplied) | Rubus phaeocarpus (pro parte misapplied) | Rubus riddelsdellii (pro parte misapplied) | Rubus rubritinctus (pro parte misapplied) | Rubus ulmifolius var. ulmifolius (pro parte misapplied) | Rubus anglocandicans (misapplied) | Rubus laudatus (misapplied)],Rubus,species,APC,accepted,Rubus anglocandicans A.Newton,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Phaius tancarvilleae,Phaius tancarvilleae,Phaius amboinensis,Phaius amboinensis [alternative possible names: Phaius australis (pro parte misapplied) | Phaius bernaysii (pro parte misapplied) | Phaius australis (misapplied) | Phaius bernaysii (misapplied)],Phaius,species,APC,accepted,Phaius amboinensis Blume,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Cuscuta racemosa,Cuscuta racemosa,Cuscuta epithymum,Cuscuta epithymum [alternative possible names: Cuscuta suaveolens (pro parte misapplied)],Cuscuta,species,APC,accepted,Cuscuta epithymum (L.) L.,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Rubus chloocladus,Rubus chloocladus,Rubus anglocandicans,Rubus anglocandicans [alternative possible names: Rubus leucostachys (pro parte misapplied) | Rubus vestitus (pro parte misapplied) | Rubus leucostachys (misapplied)],Rubus,species,APC,accepted,Rubus anglocandicans A.Newton,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Festuca elatior,Festuca elatior,Lolium arundinaceum,Lolium arundinaceum [alternative possible names: Lolium pratense (pro parte misapplied)],Lolium,species,APC,accepted,Lolium arundinaceum (Schreb.) Darbysh.,Exact match of taxon name to an APC-known canonical name once punctuation and filler words are removed (2024-08-16),pro parte misapplied,1
Picris hieracioides,Picris hieracioides,Picris angustifolia subsp. merxmuelleri,Picris angustifolia subsp. merxmuelleri [

### Reproducibility

Here are details about the version of the `APCalign` package used here:

In [12]:
packageVersion("APCalign")
default_version()

[1] ‘1.0.2’

[1] "2024-07+31"

In [13]:
citation("APCalign")

To cite package ‘APCalign’ in publications use:

  Wenk E, Cornwell W, Fuchs A, Kar F, Monro A, Sauquet H, Stephens R,
  Falster D (2024). “APCalign: an R package workflow and app for
  aligning and updating flora names to the Australian Plant Census.”
  _Australian Journal of Botany_. R package version: 1.0.1,
  <https://www.biorxiv.org/content/10.1101/2024.02.02.578715v1>.

A BibTeX entry for LaTeX users is

  @Article{,
    title = {APCalign: an R package workflow and app for aligning and updating flora names to the Australian Plant Census},
    journal = {Australian Journal of Botany},
    author = {Elizabeth Wenk and Will Cornwell and Ann Fuchs and Fonti Kar and Anna Monro and Herve Sauquet and Ruby Stephens and Daniel Falster},
    year = {2024},
    note = {R package version: 1.0.1},
    url = {https://www.biorxiv.org/content/10.1101/2024.02.02.578715v1},
  }

## Check taxonomy with MCVP

In [28]:

library(rWCVP)

In [31]:
matched_table <- wcvp_match_names(head(all_traits), name_col="species")



── Matching names to WCVP ──────────────────────────────────────────────────────

ℹ Using the `species` column

! No author information supplied - matching on taxon name only



── Exact matching  names ──



✔ Found 6 of  names



── Matching complete! ──



✔ Matched 6 of 6 names

ℹ Exact (without author): 6

! Names with multiple matches: 0



In [32]:
str(matched_table)

'data.frame':	6 obs. of  36 variables:
 $ family             : chr  "Brassicaceae" "Myrtaceae" "Myrtaceae" "Apiaceae" ...
 $ genus              : chr  "Lepidium" "Eucalyptus" "Melaleuca" "Actinotus" ...
 $ spp                : num  2358 2359 2360 2361 2362 ...
 $ species            : chr  "Lepidium oxytrichum" "Eucalyptus williamsiana" "Melaleuca glomerata" "Actinotus helianthi" ...
 $ current_spp        : num  2358 2359 2360 2361 2362 ...
 $ current_species    : chr  "Lepidium oxytrichum" "Eucalyptus williamsiana" "Melaleuca glomerata" "Actinotus helianthi" ...
 $ taxonrank          : chr  "Species" "Species" "Species" "Species" ...
 $ establishment      : chr  "Alive in NSW, Native" "Alive in NSW, Native" "Alive in NSW, Native" "Alive in NSW, Native" ...
 $ current            : chr  "true" "true" "true" "true" ...
 $ nquadrat           : num  0 0 0 0 0 2
 $ germ8              : num  0 0 0 0 0 0
 $ rect2              : num  0 1 0 2 0 0
 $ germ1              : num  0 1 0 1 0 0
 $ grow1

In [41]:
fireveg_species <- all_traits |> 
    filter(!species %in% c(NA,"")) |>
    select(species) |>
    distinct()

In [43]:
matched_table <- wcvp_match_names(fireveg_species, name_col="species", progress_bar = FALSE)




Matching ■■■■                              12% | ETA:  2h

── Matching names to WCVP ──────────────────────────────────────────────────────

Matching ■■■■                              12% | ETA:  2h

ℹ Using the `species` column

Matching ■■■■                              12% | ETA:  2h

! No author information supplied - matching on taxon name only

Matching ■■■■                              12% | ETA:  2h



Matching ■■■■                              12% | ETA:  2h

── Exact matching  names ──

Matching ■■■■                              12% | ETA:  2h



Matching ■■■■                              12% | ETA:  2h

✔ Found 11718 of  names

Matching ■■■■                              12% | ETA:  2h



Matching ■■■■                              12% | ETA:  2h

── Fuzzy matching 3930 names ──

Matching ■■■■                              12% | ETA:  2h



Matching ■■■■                              12% | ETA:  2h

✔ Found 1405 of 3930 names

Matching ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■  100% | 

In [44]:
s

ERROR: Error in eval(expr, envir, enclos): object 's' not found
